In [1]:
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()
from selenium import webdriver
driver= webdriver.Chrome()
import time  
from bs4 import BeautifulSoup
import sys 
import math

In [11]:
print("=" *100)
print(" 이 크롤러는 RISS 사이트의 논문 및 학술자료 수집용 웹크롤러입니다.")
print("=" *100)
print("\n")
query_txt = input('1.수집할 자료의 키워드는 무엇입니까?(여러개일 경우 , 로 구분하여 입력): ')
chrome_path = "c:/temp/chromedriver_91/chromedriver.exe"
driver = webdriver.Chrome(chrome_path)
url = 'http://www.riss.kr/'
driver.get(url)
time.sleep(2)

print("2. 이 키워드로 아래의 장르 중 어떤 장르의 정보를 수집할까요?")
print("\n")
print("1.학위논문"," "*5,"2.국내학술논문"," "*5,"3.해외학술논문"," "*5,"4.학술지")
print("5.단행본"," "*5,"6.공개강의"," "*5,"7.연구보고서")
print("\n")
genre = int(input("위 장르 중 수집할 장르의 번호를 입력하세요: "))
print("\n")
element = driver.find_element_by_id("query")
driver.find_element_by_id("query").click()
element.send_keys(query_txt)
element.send_keys("\n")
driver.find_element_by_link_text("국내학술논문").click()
time.sleep(2)

fc_name = input('3.결과를 저장할 csv형식의 파일명을 쓰세요(예: c:\\temp\\riss.csv): ')
fx_name = input('4.결과를 저장할 xls형식의 파일명을 쓰세요(예: c:\\temp\\riss.xls): ')
print("\n")

html_1 = driver.page_source
soup = BeautifulSoup(html_1,'html.parser')
total = soup.find('div','searchBox')('span')[1].text
print(f"{query_txt} 키워드로 국내학술논문 부분에서 검색된 자료의 건수는 총 {total} 건 입니다.")
extract = int(input("이 중에서 크롤링 할 건수는 몇건입니까?"))
print(f"{query_txt} 키워드로 국내학술논문을 검색하여 총 {total} 건 중 {extract}건의 정보를 수집하겠습니다")



 이 크롤러는 RISS 사이트의 논문 및 학술자료 수집용 웹크롤러입니다.


1.수집할 자료의 키워드는 무엇입니까?(여러개일 경우 , 로 구분하여 입력): 해양자원
2. 이 키워드로 아래의 장르 중 어떤 장르의 정보를 수집할까요?


1.학위논문       2.국내학술논문       3.해외학술논문       4.학술지
5.단행본       6.공개강의       7.연구보고서


위 장르 중 수집할 장르의 번호를 입력하세요: 2


3.결과를 저장할 csv형식의 파일명을 쓰세요(예: c:\temp\riss.csv): cs.csv
4.결과를 저장할 xls형식의 파일명을 쓰세요(예: c:\temp\riss.xls): xl.xls


해양자원 키워드로 국내학술논문 부분에서 검색된 자료의 건수는 총 3,585 건 입니다.
이 중에서 크롤링 할 건수는 몇건입니까?30
해양자원 키워드로 국내학술논문을 검색하여 총 3,585 건 중 30건의 정보를 수집하겠습니다


In [13]:
nums = []
title = []
writer = []
institution =[]
years = []
symposium = []
url = []
page_no=[ ]
no = 1
for i in range(10,extract) :
    if i % 10 == 0 :
            page_no.append(i + 1)
for a in range(1, extract + 1) :
    html_2 = driver.page_source
    soup_2 = BeautifulSoup(html_2, 'html.parser')

    list_ = soup_2.find('div','srchResultListW').find_all('li')
    for data in list_:
        try :
            title_ = data.find('div','cont').find('p','title').get_text()
        except :
            continue
        print(f"{no}번째 정보를 추출하고 있습니다.","="*10)
        print(f"1.번호 : {no}")
        nums.append(no)
        print(f"2.제목 : {title_}")
        title.append(title_)
        print(f"3.작성자 : {data.find('span','writer').get_text()}")
        writer.append(data.find('span','writer').get_text())
        print(f"4.소속기관 : {data.find('span','assigned').get_text()}")
        institution.append(data.find('span','assigned').get_text())
        print(f"5.발표년도 : {data.find('p','etc').find_all('span')[2].get_text()}")  
        years.append(data.find('p','etc').find_all('span')[2].get_text())
        print(f"6.논문/자료집 : {data.find('p','etc').find_all('span')[3].get_text()}")  
        symposium.append(data.find('p','etc').find_all('span')[3].get_text())
        url_='http://www.riss.kr'+data.find('div','cont').find('p','title').select('a')[0]['href']
        print("7.논문 URL 주소 :",url_)  
        url.append(url_)
        print("\n")
        no += 1
        if no > extract:
            break
        time.sleep(1)        # 페이지 변경 전 1초 대기 

    c = math.floor(a/10)-1 
    a += 1
    if no > extract:
        break
    if a == page_no[c]:
        driver.find_element_by_link_text('다음 페이지로').click()
    else :
        driver.find_element_by_link_text('%s' %a).click() # 다음 페이지번호 클릭

print("요청하신 작업이 모두 완료되었습니다")

import pandas as pd 

df = pd.DataFrame()
df['번호']=nums
df['제목']=pd.Series(title)
df['저자']=pd.Series(writer)
df['소속(발행)기관']=pd.Series(institution)
df['발표년도']=pd.Series(years)
df['논문집/자료집']=pd.Series(symposium)
df['논문 URL 주소']=pd.Series(url)

# xls 형태로 저장하기
df.to_excel(fx_name,index=False, encoding="utf-8")

# csv 형태로 저장하기
df.to_csv(fc_name,index=False, encoding="utf-8-sig")

print('요청하신 데이터 수집 작업이 정상적으로 완료되었습니다')

1번째 정보를 추출하고 있습니다. ==========
1.번호 : 1
2.제목 : 발표논문 / 해양생물자원으로서 해조류 : 생물활성물질의 정제와 분자적 응용
3.작성자 : 홍용기(Yong Ki Hong)
4.소속기관 : 한국조류학회
5.발표년도 : 2000
6.논문/자료집 : 국제심포지움 일정 및 발표논문집 - 21세기, 해양환경과 해양생물자원의 전망
7.논문 URL 주소 : http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=508c47ca1906d5ecffe0bdc3ef48d419


2번째 정보를 추출하고 있습니다. ==========
1.번호 : 2
2.제목 : 수거된 해양폐기물 자원화 기술 개발(Ⅰ)
3.작성자 : 길상인(Sang-In Keel), 윤진한(Jin-Han Yun), 최연석(Yeon-Seok Choi), 강창구(Chang-Gu Kang), 유정석(Jeong-Seok Yu)
4.소속기관 : 한국해양환경·에너지학회
5.발표년도 : 2002
6.논문/자료집 : 한국해양환경·에너지학회지
7.논문 URL 주소 : http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=09ec1e3541fcc118ffe0bdc3ef48d419


3번째 정보를 추출하고 있습니다. ==========
1.번호 : 3
2.제목 : 해양경찰 미래 발전을 위한 인적 자원 확보 방안연구
3.작성자 : 윤병두
4.소속기관 : 한국해양경찰학회
5.발표년도 : 2021
6.논문/자료집 : 한국해양경찰학회보
7.논문 URL 주소 : http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=fcc6dea7df1a2c0fd18150b21a227875


4번째 정보를 추출하고 있습니다. ===

27번째 정보를 추출하고 있습니다. ==========
1.번호 : 27
2.제목 : 기후변화가 해양생물자원에 미치는 영향 : 체제 전환(regime shift)과 일본에서의 연구 현황을 중심으로
3.작성자 : 원남일
4.소속기관 : 한국해양수산개발원
5.발표년도 : 2011
6.논문/자료집 : 계간 해양수산
7.논문 URL 주소 : http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=e90c74faa6eaa462ffe0bdc3ef48d419


28번째 정보를 추출하고 있습니다. ==========
1.번호 : 28
2.제목 : 해양 및 수산자원 관리를 위한 해양경계정보 구축방안에 관한 연구
3.작성자 : 김종규(Jong-Kyu KIM), 김정현(Jung-Hyun KIM)
4.소속기관 : 한국수산해양교육학회
5.발표년도 : 2005
6.논문/자료집 : 水産海洋敎育硏究
7.논문 URL 주소 : http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=d487e80bc91ebb93ffe0bdc3ef48d419


29번째 정보를 추출하고 있습니다. ==========
1.번호 : 29
2.제목 : 未來의 海洋資源 開發
3.작성자 : 金成貴
4.소속기관 : 한국해양수산개발원
5.발표년도 : 1990
6.논문/자료집 : 해양정책연구
7.논문 URL 주소 : http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=e053c1568cdb1f9cffe0bdc3ef48d419


30번째 정보를 추출하고 있습니다. ==========
1.번호 : 30
2.제목 : 국가 해수자원으로서 해양심층수의 공적 관리를 위한 법제적 논의
3.작성자 : 안기수(Kisoo AHN)
4.소속기관 :